ASAP CRN Metadata validation - wave 1

# Team Hafler. ASAP CRN Metadata validation - wave 1

Teams Hardy, Hafler, and Lee need some minor rationalization.


28 Nov 2023
Andy Henrie




In [1]:
import pandas as pd

from pathlib import Path

from utils.io import read_meta_table, get_dtypes_dict, ReportCollector
from utils.qcutils import prep_table, validate_table, validate_file, read_ASAP_CDE, reorder_table_to_CDE



%load_ext autoreload
%autoreload 2


Streamlit NOT successfully


## Clean V1 Table
write clean metadata tables according to CDE v1

## Team Hafler

In [2]:
## convert to seurat Object
data_path = Path.home() / ("Projects/ASAP")
metadata_path = data_path / "team-hafler/metadata"

sheets = ["SAMPLE","SUBJECT","CLINPATH","STUDY","PROTOCOL"]
excel_path = metadata_path / "ASAP_CDE_ALL_Team_Hafler_v1.xlsx"
STUDY = pd.read_excel(excel_path,sheet_name="STUDY",header=1).drop(columns="Field")
CLINPATH = pd.read_excel(excel_path,sheet_name="CLINPATH",header=1).drop(columns="Field")
SUBJECT = pd.read_excel(excel_path,sheet_name="SUBJECT",header=1).drop(columns="Field")
SAMPLE = pd.read_excel(excel_path,sheet_name="SAMPLE",header=1).drop(columns="Field")
PROTOCOL = pd.read_excel(excel_path,sheet_name="PROTOCOL",header=1).drop(columns="Field")
metadata_path

PosixPath('/Users/ergonyc/Projects/ASAP/team-hafler/metadata')

In [3]:

metadata_version = "v1"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

read url


In [4]:

def add_hafler_batch(sample_df):

    # First batch: HSDG07HC HSDG10HC HSDG148PD HSDG199PD
    # batch[batch.sample_id in ['hSDG07HC', 'hSDG10HC', 'hSDG148PD', 'hSDG199PD']]=1
    Batch_1 = ['hSDG07', 'hSDG10', 'hSDG148', 'hSDG199'] 
    # Second batch: hsDG101HC hsDG13HC hsDG151PD hsDG197PD hsDG30HC hsDG99HC
    Batch_2 = ['hSDG101', 'hSDG13', 'hSDG151', 'hSDG197', 'hSDG30', 'hSDG99']
    # Third batch: hsDG142PD hsDG208PD
    Batch_3 = ['hSDG142', 'hSDG208'] 


    batch_col = []
    for row in sample_df.sample_id:
        if row in Batch_1:
            batch_col.append("Batch_1")
        elif row in Batch_2:
            batch_col.append("Batch_2")
        elif row in Batch_3:
            batch_col.append("Batch_3")
        else:
            print("ERROR >>>>>>>> not no batch info")
            batch_col.append("")


    sample_df['batch'] = batch_col
    return sample_df

SAMPLE = add_hafler_batch(SAMPLE)

In [5]:
# fix replicate & replicate_count
SAMPLE.replace("Nan", "", inplace=True)

SAMPLE['replicate'] = "Rep1"
SAMPLE['replicate_count'] = 1

SAMPLE.head()

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id
0,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hSDG101,hSDG101,HC03,Rep1,1,0,Batch_2,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hSDG101,hSDG101,HC03,Rep1,1,0,Batch_2,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:

STUDY = prep_table(STUDY, CDE_df)
STUDY['metadata_version_date']

0    V1_08/27/2023
Name: metadata_version_date, dtype: object

In [7]:
# update metadata_version_date
STUDY['metadata_version_date'] = METADATA_VERSION_DATE

In [8]:
report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()


🚨⚠️❗ **Missing Required Fields in STUDY: submitter_email**
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [9]:
report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()


All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [10]:
SUBJECT.replace("Nan", "", inplace=True)
# Testing the function with SUBJECT.csv and CDE.csv
report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 12/12 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*sex*_:  invalid values 💩'F', 'M'
    - valid ➡️ 'Male', 'Female', 'Intersex', 'Unnown'
- _*race*_:  invalid values 💩'B', 'W'
    - valid ➡️ 'American Indian or Alaska Native', 'Asian', 'White', 'Black or African American', 'Multi-racial', 'Native Hawaiian or Other Pacific Islander', 'Other', 'Unknown', 'Not Reported'
- _*ethnicity*_:  invalid values 💩'B', 'W'
    - valid ➡️ 'Hispanic or Latino', 'Not Hispanic or Latino', 'Unknown', 'Not Reported'
- _*primary_diagnosis*_:  invalid values 💩'Normal control', 'Idiopathic Parkinson's disease'
    - valid ➡️ 'Healthy Control', 'Idiopathic PD', 'Alzheimer’s disease', 'Frontotemporal dementia', 'Corticobasal syndrome', 'Dementia with Lewy bodies', 'Dopa-responsive dystonia', 'Essential tremor', 'Hemiparkinson/he

In [11]:

SUBJECT = prep_table(SUBJECT, CDE_df)


In [12]:
SUBJECT['sex'] = SUBJECT['sex'].replace({'F':"Female", 'M':"Male"})
SUBJECT['race'] = SUBJECT['race'].replace({'W':"White", 'B':"Black or African American"})

SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace({'Normal control':"Healthy Control", "Idiopathic Parkinson's disease":"Idiopathic PD"})
report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 12/12 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*ethnicity*_:  invalid values 💩'B', 'W'
    - valid ➡️ 'Hispanic or Latino', 'Not Hispanic or Latino', 'Unknown', 'Not Reported'



In [13]:
# they "race" was reported under ethnicity, so code as "Not Reported"
SUBJECT['ethnicity'] = 'Not Reported'

In [14]:

report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 12/12 empty rows
## Enum fields have valid values in SUBJECT. 🥳



In [15]:
SAMPLE = prep_table(SAMPLE,  CDE_df)


In [16]:
# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

# set time == 0 for all samples
SAMPLE['time'] = 0

SAMPLE['file_type'] = SAMPLE['file_type'].replace({"Fastq":"fastq"})


In [17]:

# need to join with subject to get "sex" and convert to ontology term
SAMPLE_SUBJECT = SAMPLE.merge(SUBJECT, on='subject_id',  how='left')
SAMPLE_og = SAMPLE.copy()
SAMPLE['sex_ontology_term_id'] = SAMPLE_SUBJECT['sex'].replace({"Male":"PATO:0000384 (male)", "Female":"PATO:0000383 (female)" })

# ignore development_stage_ontology_term_id, self_reported_ethnicity_ontology_term_id, assay_ontology_term_id, etc for now. (Check wiht Le)

In [18]:
# fix assay
SAMPLE['assay'] = SAMPLE['assay'].replace({'v3.1 - Single Index, 10x Genomics ':"v3.1 - Single Index"})
# fix assay
SAMPLE['sequencing_length'] = SAMPLE['sequencing_length'].replace({'150bp x2':"150"})

#TODO: add file_description I1, R1, R2  "Raw sequencing data"

In [19]:

report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()


All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 36/36 empty rows

	- RIN: 36/36 empty rows

	- suspension_type: 36/36 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- pm_PH: 36/36 empty rows
## Enum fields have valid values in SAMPLE. 🥳



In [20]:
# CLINPATH.replace("Nan", "", inplace=True)
# CLINPATH.replace("nan", "", inplace=True)
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


🚨⚠️❗ **Missing Required Fields in CLINPATH: path_thal**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 12/12 empty rows

	- age_at_diagnosis: 12/12 empty rows

	- first_motor_symptom: 12/12 empty rows

	- path_year_death: 12/12 empty rows

	- brain_weight: 12/12 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 12/12 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*region_level_2*_:  invalid values 💩'Prefrontal cortex'
    - valid ➡️ 'Superior frontal gyrus', 'Middle frontal gyrus', 'Inferior frontal gyrus', 'Superior temporal gyrus', 'Middle temporal gyrus', 'Inferior temporal gyrus', 'Fusiform gyrus', 'Transentorhinal region', 'Entorinal region', 'Subiculum', 'CA1-CA4', 'Amygdala', 'Periamygdala cortex', 'Anterior cingulate gyrus', 'Posterior cingulate gyrus', 'Superior parietal lobule', 'Inferior parietal lobule', 'Parastriate cortex', 'Peristriate cortex', 'Striate cortex', 'Insular cortex', 'Caudate nucleus', 'Putamen', 'Globus pa

In [21]:
CLINPATH = prep_table(CLINPATH, CDE_df)


In [22]:
# redact "Prefrontal Cortex" from region_level_2 for now
CLINPATH['region_level_2'] = CLINPATH['region_level_2'].replace({'Prefrontal Cortex':""})

# leave te APOE_e4_status as is for now . multiple are coded as "2,3" 
# but remove commas
CLINPATH["APOE_e4_status"] = CLINPATH["APOE_e4_status"].str.replace(",","")

# need to fix the path_autopsy_dx_main

In [23]:
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()

🚨⚠️❗ **Missing Required Fields in CLINPATH: path_thal**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 12/12 empty rows

	- age_at_diagnosis: 12/12 empty rows

	- first_motor_symptom: 12/12 empty rows

	- path_year_death: 12/12 empty rows

	- brain_weight: 12/12 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 12/12 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*region_level_2*_:  invalid values 💩'Prefrontal cortex'
    - valid ➡️ 'Superior frontal gyrus', 'Middle frontal gyrus', 'Inferior frontal gyrus', 'Superior temporal gyrus', 'Middle temporal gyrus', 'Inferior temporal gyrus', 'Fusiform gyrus', 'Transentorhinal region', 'Entorinal region', 'Subiculum', 'CA1-CA4', 'Amygdala', 'Periamygdala cortex', 'Anterior cingulate gyrus', 'Posterior cingulate gyrus', 'Superior parietal lobule', 'Inferior parietal lobule', 'Parastriate cortex', 'Peristriate cortex', 'Striate cortex', 'Insular cortex', 'Caudate nucleus', 'Putamen', 'Globus pa

In [24]:
data_path = Path.home() / "Projects/ASAP/team-hafler"


In [25]:
# fix the column order
STUDY = reorder_table_to_CDE(STUDY, "STUDY", CDE_df)
SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE_df)
PROTOCOL = reorder_table_to_CDE(PROTOCOL, "PROTOCOL", CDE_df)
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE_df)     
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)

# write the clean metadata
STUDY.to_csv(data_path / "metadata/STUDY.csv")
PROTOCOL.to_csv(data_path / "metadata/PROTOCOL.csv")
CLINPATH.to_csv(data_path / "metadata/CLINPATH.csv")
SAMPLE.to_csv(data_path / "metadata/SAMPLE.csv")
SUBJECT.to_csv(data_path / "metadata/SUBJECT.csv")

# also writh them to clean...
# 
#  

export_root = Path.cwd() / "clean/team-Hafler"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")


added empty column submitter_email to STUDY table
added empty column DV200 to SAMPLE table
added empty column path_thal to CLINPATH table
added empty column dig_slide_avail to CLINPATH table
added empty column quant_path_avail to CLINPATH table


In [26]:
# STUDY.to_csv( metadata_path / "STUDY.csv")
# CLINPATH.to_csv( metadata_path / "CLINPATH.csv")
# SUBJECT.to_csv( metadata_path / "SUBJECT.csv")
# SAMPLE.to_csv( metadata_path / "SAMPLE.csv")
# PROTOCOL.to_csv( metadata_path / "PROTOCOL.csv")

## Update the table to v2

In [27]:
from update_schema import update_tables_to_CDEv2

tables_path = Path.cwd() / "clean/team-Hafler"


CDEv1 = pd.read_csv( Path.cwd() / "ASAP_CDE_v1.csv" )
CDEv2 = pd.read_csv( Path.cwd() / "ASAP_CDE_v2.csv" )


STUDYv2, PROTOCOLv2, SAMPLEv2, SUBJECTv2, CLINPATHv2, DATAv2 = update_tables_to_CDEv2(tables_path, CDEv1, CDEv2)


In [28]:

export_root = Path.cwd() / "clean/team-Hafler/v2"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


In [29]:

STUDYv2.to_csv( export_root / "STUDY.csv")
PROTOCOLv2.to_csv(export_root / "PROTOCOL.csv")
SAMPLEv2.to_csv(export_root / "SAMPLE.csv")
SUBJECTv2.to_csv(export_root / "SUBJECT.csv")
CLINPATHv2.to_csv(export_root / "CLINPATH.csv")
DATAv2.to_csv(export_root / "DATA.csv")


Transfer cleaned metadata to raw buckets 




## transfer hafler metadata Hafler 

In [6]:
# Lee
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hafler-credentials.json 


Activated service account credentials for: [raw-admin-hafler@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [10]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-hafler/metadata/v2"


      1260  2023-11-30T21:03:06Z  gs://asap-raw-data-team-hafler/metadata/v2/CLINPATH.csv#1701378186816137  metageneration=1
      5290  2023-11-30T21:03:07Z  gs://asap-raw-data-team-hafler/metadata/v2/DATA.csv#1701378187386113  metageneration=1
      1761  2023-11-30T21:03:07Z  gs://asap-raw-data-team-hafler/metadata/v2/PROTOCOL.csv#1701378187767597  metageneration=1
      3260  2023-11-30T21:03:07Z  gs://asap-raw-data-team-hafler/metadata/v2/SAMPLE.csv#1701378187201094  metageneration=1
      1446  2023-11-30T21:03:07Z  gs://asap-raw-data-team-hafler/metadata/v2/STUDY.csv#1701378187591914  metageneration=1
      1491  2023-11-30T21:03:07Z  gs://asap-raw-data-team-hafler/metadata/v2/SUBJECT.csv#1701378187002956  metageneration=1
                                 gs://asap-raw-data-team-hafler/metadata/v2/v2_20231114/
                                 gs://asap-raw-data-team-hafler/metadata/v2/v2_20231128/
TOTAL: 6 objects, 14508 bytes (14.17 KiB)


In [8]:
Path.cwd()

NameError: name 'Path' is not defined

In [9]:

# !gsutil -u dnastack-asap-parkinsons cp -r ./clean/team-Hafler/v2_20231128  "gs://asap-raw-data-team-hafler/metadata/v2"
!gsutil -u dnastack-asap-parkinsons cp -r "./clean/team-Hafler/v2_20231130/*.csv"  "gs://asap-raw-data-team-hafler/metadata/v2"


Copying file://./clean/team-Hafler/v2_20231130/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hafler/v2_20231130/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hafler/v2_20231130/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hafler/v2_20231130/DATA.csv [Content-Type=text/csv]...
- [4 files][ 11.0 KiB/ 11.0 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Hafler/v2_20231130/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hafler/v2_20231130/PROTOCOL.csv [Content-Type=text/csv]...
- [6 files][ 14.2 KiB/ 14.2 KiB]                                                
Operation completed over 6 objects/14.2 KiB.                              

### copy to workflow-dev bucket

First copy each set of metadata locally ...

In [11]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [13]:
!gsutil  cp -r "./clean/team-Hafler/v2_20231130/*.csv"  "gs://asap-workflow-dev/metadata/v2/hafler"

Copying file://./clean/team-Hafler/v2_20231130/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hafler/v2_20231130/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hafler/v2_20231130/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hafler/v2_20231130/DATA.csv [Content-Type=text/csv]...
- [4 files][ 11.0 KiB/ 11.0 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Hafler/v2_20231130/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hafler/v2_20231130/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][ 14.2 KiB/ 14.2 KiB]                                                
Operation completed over 6 objects/14.2 KiB.                              

In [5]:
print("damn")

damn


## check file md5s

In [31]:
from utils.checksums import extract_md5_from_details, extract_md5_from_details2


In [37]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hafler-credentials.json  



Activated service account credentials for: [raw-admin-hafler@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [32]:

# !gcloud storage hash "gs://asap-raw-data-team-hardy/**/*.gz"  --skip-crc32c --hex  --billing-project dnastack-asap-parkinsons > hardy_hexhash.log

!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-hafler/**/*.gz" > hafler_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [46]:
bucket_files_md5 = extract_md5_from_details2("hafler_hexhash.log")



checksum = DATAv2[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)



/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_56418/1240943610.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] =checksum['file_MD5'].str.strip()
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_56418/1240943610.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)


In [48]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()
#empty means success!!


[]